# Week 2: https://www.coursera.org/learn/natural-language-processing-tensorflow/home/week/2

In [1]:
# Embeddings: words and associated words are clustered in 
# vectors in multi-dimensional space

In [2]:
# IMDB classification

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.1.0'

In [6]:
# !pip install -q tensorflow-datasets

In [5]:
import tensorflow_datasets as tfds

## Load in data

In [6]:
imbd, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [ ]:
# train test split

In [7]:
train, test = imbd['train'], imbd['test']

In [8]:
train_sentences = []
train_labels = []

for i,j in train:
    train_sentences.append(str(i.numpy()))
    train_labels.append(str(j.numpy()))

In [9]:
test_sentences = []
test_labels = []

for i,j in test:
    test_sentences.append(str(i.numpy()))
    test_labels.append(str(j.numpy()))

In [10]:
import numpy as np
train_labels_array = np.array(train_labels)
test_labels_array = np.array(test_labels)

### convert str label to int

In [11]:
map_str_labels_to_int = {'1':1, '0':0}

In [12]:
train_labels_int=list(map(int, train_labels))
test_labels_int=list(map(int, test_labels))

In [13]:
train_labels_array_int = np.array(train_labels_int)
test_labels_array_int = np.array(test_labels_int)

## Create tokenizer and apply padding

In [14]:
VOCAB_SIZE = int(1e+4)  # size of vocabulary
OOV_TOK = "<OOV>"
MAX_LEN = 120  # max number of items in sequence
TRUNC_TYPE = 'post'
EMBED_DIM = 16  # embedding dimension (= output)

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer,text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [16]:
tokenizer = Tokenizer(oov_token=OOV_TOK, num_words=VOCAB_SIZE)

In [17]:
tokenizer.fit_on_texts(train_sentences)

In [18]:
word_index = tokenizer.word_index # 86539 length

In [20]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)

In [21]:
train_padded = pad_sequences(train_sequences, maxlen=MAX_LEN, truncating=TRUNC_TYPE)

In [22]:
train_padded.shape

(25000, 120)

In [23]:
# apply on test

In [24]:
test_sequences = tokenizer.texts_to_sequences(test_sentences)

In [25]:
test_padded = pad_sequences(test_sequences, maxlen=MAX_LEN)

In [26]:
test_padded.shape

(25000, 120)

### Reverse word index tuple

In [89]:
word_index_reversed = dict([(value,key) for (key,value) in word_index.items()])

## Model

### With flatten layer

In [28]:
model = tf.keras.Sequential([
    
    # Result is 2D array with length of sentence and dimension of embedding
    tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, 
                                  output_dim=EMBED_DIM,
                                  input_length=MAX_LEN),
    
    # In NLP another layer type than flatten is used (due to size of output vector)
    tf.keras.layers.Flatten(),
    
    # Dense NN for classification
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [29]:
(
    model
    .compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


### With global average pooling layer

In [30]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, 
                                  output_dim=EMBED_DIM,
                                  input_length=MAX_LEN),
    
    # Averages across the vectors to flatten it out (bit faster)
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [31]:
(
    model
    .compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


### Fit model and validate

In [32]:
NUM_EPOCHS = 10

In [33]:
(
    model
    .fit(x=train_padded,
         y=train_labels_array_int,
         epochs=NUM_EPOCHS,
         validation_data=(test_padded, test_labels_array_int)
        )
)

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - ETA: 12:48 - loss: 0.6930 - accuracy: 0.531 - ETA: 1:29 - loss: 0.6932 - accuracy: 0.534 - ETA: 52s - loss: 0.6929 - accuracy: 0.5195 - ETA: 36s - loss: 0.6929 - accuracy: 0.509 - ETA: 28s - loss: 0.6925 - accuracy: 0.518 - ETA: 23s - loss: 0.6923 - accuracy: 0.522 - ETA: 20s - loss: 0.6919 - accuracy: 0.531 - ETA: 18s - loss: 0.6917 - accuracy: 0.531 - ETA: 16s - loss: 0.6914 - accuracy: 0.543 - ETA: 15s - loss: 0.6909 - accuracy: 0.561 - ETA: 14s - loss: 0.6903 - accuracy: 0.572 - ETA: 13s - loss: 0.6898 - accuracy: 0.583 - ETA: 12s - loss: 0.6890 - accuracy: 0.595 - ETA: 11s - loss: 0.6886 - accuracy: 0.600 - ETA: 11s - loss: 0.6881 - accuracy: 0.605 - ETA: 10s - loss: 0.6873 - accuracy: 0.610 - ETA: 10s - loss: 0.6867 - accuracy: 0.615 - ETA: 9s - loss: 0.6859 - accuracy: 0.620 - ETA: 9s - loss: 0.6851 - accuracy: 0.62 - ETA: 9s - loss: 0.6844 - accuracy: 0.63 - ETA: 8s - los

25000/25000 [==============================] - ETA: 14s - loss: 0.2727 - accuracy: 0.906 - ETA: 7s - loss: 0.3183 - accuracy: 0.883 - ETA: 6s - loss: 0.3334 - accuracy: 0.88 - ETA: 6s - loss: 0.3465 - accuracy: 0.85 - ETA: 6s - loss: 0.3447 - accuracy: 0.86 - ETA: 6s - loss: 0.3475 - accuracy: 0.86 - ETA: 6s - loss: 0.3473 - accuracy: 0.86 - ETA: 5s - loss: 0.3482 - accuracy: 0.86 - ETA: 5s - loss: 0.3457 - accuracy: 0.86 - ETA: 5s - loss: 0.3486 - accuracy: 0.86 - ETA: 5s - loss: 0.3454 - accuracy: 0.86 - ETA: 5s - loss: 0.3465 - accuracy: 0.86 - ETA: 5s - loss: 0.3490 - accuracy: 0.86 - ETA: 5s - loss: 0.3488 - accuracy: 0.85 - ETA: 5s - loss: 0.3470 - accuracy: 0.86 - ETA: 5s - loss: 0.3479 - accuracy: 0.85 - ETA: 5s - loss: 0.3497 - accuracy: 0.85 - ETA: 5s - loss: 0.3511 - accuracy: 0.85 - ETA: 5s - loss: 0.3513 - accuracy: 0.85 - ETA: 5s - loss: 0.3532 - accuracy: 0.85 - ETA: 4s - loss: 0.3525 - accuracy: 0.85 - ETA: 4s - loss: 0.3523 - accuracy: 0.85 - ETA: 4s - loss: 0.3516 - a

25000/25000 [==============================] - ETA: 8s - loss: 0.2772 - accuracy: 0.93 - ETA: 7s - loss: 0.3285 - accuracy: 0.87 - ETA: 6s - loss: 0.3151 - accuracy: 0.87 - ETA: 7s - loss: 0.3024 - accuracy: 0.87 - ETA: 7s - loss: 0.3083 - accuracy: 0.87 - ETA: 7s - loss: 0.3225 - accuracy: 0.86 - ETA: 7s - loss: 0.3250 - accuracy: 0.86 - ETA: 7s - loss: 0.3160 - accuracy: 0.86 - ETA: 7s - loss: 0.3107 - accuracy: 0.87 - ETA: 6s - loss: 0.2972 - accuracy: 0.88 - ETA: 6s - loss: 0.2910 - accuracy: 0.88 - ETA: 6s - loss: 0.2878 - accuracy: 0.88 - ETA: 6s - loss: 0.2852 - accuracy: 0.88 - ETA: 5s - loss: 0.2844 - accuracy: 0.88 - ETA: 5s - loss: 0.2801 - accuracy: 0.88 - ETA: 5s - loss: 0.2820 - accuracy: 0.88 - ETA: 5s - loss: 0.2857 - accuracy: 0.88 - ETA: 5s - loss: 0.2888 - accuracy: 0.88 - ETA: 5s - loss: 0.2863 - accuracy: 0.88 - ETA: 5s - loss: 0.2872 - accuracy: 0.88 - ETA: 5s - loss: 0.2884 - accuracy: 0.88 - ETA: 5s - loss: 0.2889 - accuracy: 0.88 - ETA: 5s - loss: 0.2861 - accu

25000/25000 [==============================] - ETA: 11s - loss: 0.2189 - accuracy: 0.937 - ETA: 6s - loss: 0.2231 - accuracy: 0.933 - ETA: 7s - loss: 0.2315 - accuracy: 0.91 - ETA: 6s - loss: 0.2166 - accuracy: 0.92 - ETA: 6s - loss: 0.2087 - accuracy: 0.92 - ETA: 6s - loss: 0.2176 - accuracy: 0.92 - ETA: 6s - loss: 0.2260 - accuracy: 0.92 - ETA: 5s - loss: 0.2219 - accuracy: 0.92 - ETA: 5s - loss: 0.2178 - accuracy: 0.92 - ETA: 5s - loss: 0.2199 - accuracy: 0.92 - ETA: 5s - loss: 0.2277 - accuracy: 0.92 - ETA: 5s - loss: 0.2276 - accuracy: 0.91 - ETA: 5s - loss: 0.2290 - accuracy: 0.91 - ETA: 5s - loss: 0.2314 - accuracy: 0.91 - ETA: 5s - loss: 0.2347 - accuracy: 0.91 - ETA: 5s - loss: 0.2343 - accuracy: 0.91 - ETA: 5s - loss: 0.2311 - accuracy: 0.91 - ETA: 5s - loss: 0.2302 - accuracy: 0.91 - ETA: 5s - loss: 0.2343 - accuracy: 0.91 - ETA: 5s - loss: 0.2361 - accuracy: 0.91 - ETA: 5s - loss: 0.2345 - accuracy: 0.91 - ETA: 5s - loss: 0.2358 - accuracy: 0.91 - ETA: 5s - loss: 0.2348 - a

25000/25000 [==============================] - ETA: 11s - loss: 0.2297 - accuracy: 0.906 - ETA: 15s - loss: 0.1896 - accuracy: 0.937 - ETA: 11s - loss: 0.1784 - accuracy: 0.937 - ETA: 10s - loss: 0.1743 - accuracy: 0.942 - ETA: 9s - loss: 0.1657 - accuracy: 0.940 - ETA: 8s - loss: 0.1699 - accuracy: 0.93 - ETA: 7s - loss: 0.1763 - accuracy: 0.93 - ETA: 7s - loss: 0.1803 - accuracy: 0.93 - ETA: 7s - loss: 0.1887 - accuracy: 0.93 - ETA: 6s - loss: 0.1924 - accuracy: 0.92 - ETA: 6s - loss: 0.1918 - accuracy: 0.92 - ETA: 6s - loss: 0.1914 - accuracy: 0.92 - ETA: 6s - loss: 0.1923 - accuracy: 0.92 - ETA: 6s - loss: 0.1874 - accuracy: 0.93 - ETA: 5s - loss: 0.1871 - accuracy: 0.93 - ETA: 5s - loss: 0.1865 - accuracy: 0.93 - ETA: 5s - loss: 0.1876 - accuracy: 0.93 - ETA: 5s - loss: 0.1870 - accuracy: 0.93 - ETA: 5s - loss: 0.1865 - accuracy: 0.93 - ETA: 5s - loss: 0.1840 - accuracy: 0.93 - ETA: 5s - loss: 0.1838 - accuracy: 0.93 - ETA: 5s - loss: 0.1836 - accuracy: 0.93 - ETA: 5s - loss: 0.18

25000/25000 [==============================] - ETA: 6s - loss: 0.1768 - accuracy: 0.90 - ETA: 6s - loss: 0.1559 - accuracy: 0.94 - ETA: 6s - loss: 0.1721 - accuracy: 0.93 - ETA: 6s - loss: 0.1720 - accuracy: 0.94 - ETA: 6s - loss: 0.1639 - accuracy: 0.94 - ETA: 5s - loss: 0.1666 - accuracy: 0.94 - ETA: 5s - loss: 0.1698 - accuracy: 0.93 - ETA: 5s - loss: 0.1737 - accuracy: 0.93 - ETA: 5s - loss: 0.1813 - accuracy: 0.93 - ETA: 5s - loss: 0.1775 - accuracy: 0.93 - ETA: 5s - loss: 0.1746 - accuracy: 0.93 - ETA: 5s - loss: 0.1717 - accuracy: 0.93 - ETA: 5s - loss: 0.1698 - accuracy: 0.93 - ETA: 5s - loss: 0.1706 - accuracy: 0.93 - ETA: 5s - loss: 0.1704 - accuracy: 0.93 - ETA: 5s - loss: 0.1692 - accuracy: 0.93 - ETA: 5s - loss: 0.1714 - accuracy: 0.93 - ETA: 4s - loss: 0.1701 - accuracy: 0.93 - ETA: 4s - loss: 0.1704 - accuracy: 0.93 - ETA: 4s - loss: 0.1704 - accuracy: 0.93 - ETA: 4s - loss: 0.1695 - accuracy: 0.93 - ETA: 4s - loss: 0.1702 - accuracy: 0.93 - ETA: 4s - loss: 0.1700 - accu

25000/25000 [==============================] - ETA: 10s - loss: 0.1093 - accuracy: 0.968 - ETA: 6s - loss: 0.1893 - accuracy: 0.941 - ETA: 6s - loss: 0.1676 - accuracy: 0.94 - ETA: 6s - loss: 0.1559 - accuracy: 0.94 - ETA: 6s - loss: 0.1456 - accuracy: 0.95 - ETA: 6s - loss: 0.1582 - accuracy: 0.94 - ETA: 5s - loss: 0.1503 - accuracy: 0.95 - ETA: 5s - loss: 0.1506 - accuracy: 0.95 - ETA: 5s - loss: 0.1464 - accuracy: 0.95 - ETA: 5s - loss: 0.1436 - accuracy: 0.95 - ETA: 5s - loss: 0.1433 - accuracy: 0.95 - ETA: 5s - loss: 0.1424 - accuracy: 0.95 - ETA: 5s - loss: 0.1423 - accuracy: 0.95 - ETA: 5s - loss: 0.1416 - accuracy: 0.95 - ETA: 6s - loss: 0.1421 - accuracy: 0.95 - ETA: 6s - loss: 0.1419 - accuracy: 0.95 - ETA: 6s - loss: 0.1426 - accuracy: 0.95 - ETA: 5s - loss: 0.1436 - accuracy: 0.95 - ETA: 5s - loss: 0.1433 - accuracy: 0.95 - ETA: 5s - loss: 0.1417 - accuracy: 0.95 - ETA: 5s - loss: 0.1419 - accuracy: 0.95 - ETA: 5s - loss: 0.1425 - accuracy: 0.95 - ETA: 5s - loss: 0.1428 - a

25000/25000 [==============================] - ETA: 9s - loss: 0.1388 - accuracy: 0.96 - ETA: 19s - loss: 0.1361 - accuracy: 0.960 - ETA: 13s - loss: 0.1316 - accuracy: 0.965 - ETA: 11s - loss: 0.1357 - accuracy: 0.962 - ETA: 11s - loss: 0.1406 - accuracy: 0.960 - ETA: 11s - loss: 0.1414 - accuracy: 0.959 - ETA: 10s - loss: 0.1330 - accuracy: 0.959 - ETA: 9s - loss: 0.1308 - accuracy: 0.959 - ETA: 9s - loss: 0.1316 - accuracy: 0.96 - ETA: 9s - loss: 0.1280 - accuracy: 0.96 - ETA: 10s - loss: 0.1367 - accuracy: 0.961 - ETA: 9s - loss: 0.1327 - accuracy: 0.963 - ETA: 10s - loss: 0.1316 - accuracy: 0.963 - ETA: 10s - loss: 0.1329 - accuracy: 0.964 - ETA: 10s - loss: 0.1317 - accuracy: 0.966 - ETA: 9s - loss: 0.1342 - accuracy: 0.965 - ETA: 9s - loss: 0.1331 - accuracy: 0.96 - ETA: 9s - loss: 0.1327 - accuracy: 0.96 - ETA: 8s - loss: 0.1353 - accuracy: 0.96 - ETA: 8s - loss: 0.1326 - accuracy: 0.96 - ETA: 8s - loss: 0.1314 - accuracy: 0.96 - ETA: 8s - loss: 0.1296 - accuracy: 0.96 - ETA: 7

25000/25000 [==============================] - ETA: 5s - loss: 0.0539 - accuracy: 1.00 - ETA: 4s - loss: 0.1084 - accuracy: 0.96 - ETA: 4s - loss: 0.1063 - accuracy: 0.97 - ETA: 4s - loss: 0.1176 - accuracy: 0.96 - ETA: 5s - loss: 0.1224 - accuracy: 0.96 - ETA: 5s - loss: 0.1292 - accuracy: 0.96 - ETA: 5s - loss: 0.1235 - accuracy: 0.96 - ETA: 6s - loss: 0.1246 - accuracy: 0.96 - ETA: 6s - loss: 0.1222 - accuracy: 0.96 - ETA: 6s - loss: 0.1225 - accuracy: 0.96 - ETA: 6s - loss: 0.1215 - accuracy: 0.96 - ETA: 6s - loss: 0.1224 - accuracy: 0.96 - ETA: 6s - loss: 0.1205 - accuracy: 0.96 - ETA: 6s - loss: 0.1247 - accuracy: 0.96 - ETA: 6s - loss: 0.1241 - accuracy: 0.96 - ETA: 6s - loss: 0.1216 - accuracy: 0.96 - ETA: 6s - loss: 0.1229 - accuracy: 0.96 - ETA: 6s - loss: 0.1227 - accuracy: 0.96 - ETA: 6s - loss: 0.1250 - accuracy: 0.96 - ETA: 6s - loss: 0.1226 - accuracy: 0.96 - ETA: 6s - loss: 0.1206 - accuracy: 0.96 - ETA: 6s - loss: 0.1202 - accuracy: 0.96 - ETA: 6s - loss: 0.1193 - accu

25000/25000 [==============================] - ETA: 5s - loss: 0.0771 - accuracy: 0.96 - ETA: 4s - loss: 0.1171 - accuracy: 0.97 - ETA: 4s - loss: 0.1115 - accuracy: 0.97 - ETA: 4s - loss: 0.1106 - accuracy: 0.97 - ETA: 4s - loss: 0.1145 - accuracy: 0.97 - ETA: 4s - loss: 0.1126 - accuracy: 0.97 - ETA: 4s - loss: 0.1086 - accuracy: 0.97 - ETA: 4s - loss: 0.1039 - accuracy: 0.97 - ETA: 4s - loss: 0.1027 - accuracy: 0.97 - ETA: 4s - loss: 0.1045 - accuracy: 0.97 - ETA: 4s - loss: 0.1059 - accuracy: 0.97 - ETA: 4s - loss: 0.1058 - accuracy: 0.97 - ETA: 4s - loss: 0.1081 - accuracy: 0.97 - ETA: 3s - loss: 0.1067 - accuracy: 0.97 - ETA: 3s - loss: 0.1057 - accuracy: 0.97 - ETA: 3s - loss: 0.1078 - accuracy: 0.97 - ETA: 3s - loss: 0.1088 - accuracy: 0.97 - ETA: 3s - loss: 0.1085 - accuracy: 0.97 - ETA: 3s - loss: 0.1088 - accuracy: 0.97 - ETA: 3s - loss: 0.1121 - accuracy: 0.96 - ETA: 3s - loss: 0.1110 - accuracy: 0.96 - ETA: 3s - loss: 0.1108 - accuracy: 0.96 - ETA: 3s - loss: 0.1111 - accu

In [59]:
model.summary()
# Params in first layer: VOCAB_SIZE * EMBED_DIM
# Embedding layer input size: MAX_LEN

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [57]:
e_0 = model.layers[0]
weight_0 = e_0.get_weights()[0]

In [58]:
weight_0.shape # shape: [VOCAB_SIZE, EMBED_DIM]

(10000, 16)

In [60]:
import io

In [61]:
out_v = io.open('vecs.tsv', 'w', encoding= 'utf-8')
out_m = io.open('meta.tsv', 'w', encoding= 'utf-8')

In [91]:
# Skip the '<OOV>'; is it always on place 1?
for word_num in range(1,VOCAB_SIZE):
    word = word_index_reversed[word_num]
    embeddings = weight_0[word_num]  # embedding layer
    out_m.write(word + "\n")
    out_v.write("\t".join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()